In [1]:
import os
import random
import numpy as np

from collections import namedtuple

from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


# Stanford Sentiment Treebank - movie reviews with fine-grained labels

In [2]:
# Stanford Sentiment Treebank - movie reviews with fine-grained labels
# https://nlp.stanford.edu/sentiment/

ST_sentence = namedtuple("Stanford_Sentiment", "id sentence")
ST_score = namedtuple("Stanford_Sentiment", "id score")

sentences = dict()
scores = dict()
train = []
dev = []
test = []

for filename in ['datasetSentences.txt','datasetSplit.txt', 'sentiment_labels.txt']:
    with open("ST/"+filename,'r') as f_input:
        for line in f_input:
            
            # skip headers
            if line.startswith("sentence_index") or line.startswith('phrase id'):
                continue
            
            # load sentences
            if filename=='datasetSentences.txt':                                
                sent_id, sentence = line.split('\t', 1)
                sentences[sent_id] = sentence.strip()
            
            # load splits
            if filename=='datasetSplit.txt':
                sent_id, split = line.split(',', 1)
                split = int(split.strip())
                if split == 1:                    
                    train.append(sent_id)
                if split == 2:                                        
                    test.append(sent_id)
                if split == 3:
                    dev.append(sent_id)
            
            # sentences_id        
            if filename=='sentiment_labels.txt':                    
                sent_id, sent_score = line.split('|', 1)
                #sent_score = float(sent_score.strip())
                sample = ST_score(sent_id, float(sent_score.strip()))
                scores[sent_id] = sent_score.strip()

# Samples and Classes/Labels

In [3]:
print("Total Nr. Samples: {}".format(len(sentences)))
print("Total Nr. Scores : {}".format(len(scores)))
print()
print("Train  : {}".format(len(train)))
print("Dev    : {}".format(len(dev)))
print("Test   : {}".format(len(test)))

Total Nr. Samples: 11855
Total Nr. Scores : 239232

Train  : 8544
Dev    : 1101
Test   : 2210


In [4]:
# built two lists with sentences and labels
x_train_data = [sentences[x] for x in train]
y_train_data = [scores[x] for x in train]

x_dev_data = [sentences[x] for x in dev]
y_dev_data = [scores[x] for x in dev]

x_test_data = [sentences[x] for x in test]
y_test_data = [scores[x] for x in test]

In [5]:
# convert list of tokens/words to indexes
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train_data)
sequences_train = tokenizer.texts_to_sequences(x_train_data)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 15337 unique tokens.


In [6]:
# get the max sentence lenght, needed for padding
max_input_lenght = max([len(x) for x in sequences_train])
max_input_lenght

49

In [7]:
# pad all the sequences of indexes to the 'max_input_lenght'
x_train_data_padded = pad_sequences(sequences_train, maxlen=max_input_lenght, padding='post', truncating='post')
x_dev_data_padded = pad_sequences(tokenizer.texts_to_sequences(x_dev_data), maxlen=max_input_lenght, padding='post', truncating='post')
x_test_data_padded = pad_sequences(tokenizer.texts_to_sequences(x_test_data), maxlen=max_input_lenght, padding='post', truncating='post')

## Transform scores to classes as re-labeled by Socher et al. (2013)
- 0 - 2.0    : very negative
- 2.0 - 4.0  : negative
- 4.0 - 6.0  : neutral
- 6.0 - 8.0  : negative
- 8.0 - 10.0 : very positive

In [8]:
def convert_to_categories(y_data):
    y_categories = []
    for score in y_data:
        if 0.0<=float(score)<0.2:
            y_categories.append('very_negative')
        elif 0.2<=float(score)<0.4:
            y_categories.append('negative')
        elif 0.4<=float(score)<0.6:
            y_categories.append('neutral')
        elif 0.6<=float(score)<0.8:
            y_categories.append('positive')
        elif 0.8<=float(score)<=1.0:
            y_categories.append('very positive')
            
    return y_categories

In [9]:
# Convert from scores to categories
y_train_data_categ = convert_to_categories(y_train_data)
y_dev_data_categ = convert_to_categories(y_dev_data)
y_test_data_categ = convert_to_categories(y_test_data)

# Encode the labels, each must be a vector with dim = num. of possible labels
le = LabelEncoder()
le.fit(y_train_data_categ)

labels_encoded_train = le.transform(y_train_data_categ)
labels_encoded_dev = le.transform(y_dev_data_categ)
labels_encoded_test = le.transform(y_test_data_categ)

categorical_labels_train = to_categorical(labels_encoded_train, num_classes=None)
categorical_labels_dev = to_categorical(labels_encoded_dev, num_classes=None)
categorical_labels_test = to_categorical(labels_encoded_test, num_classes=None)

In [10]:
print(x_train_data_padded.shape)
print(categorical_labels_train.shape)

(8544, 49)
(8544, 5)


In [11]:
print(x_dev_data_padded.shape)
print(labels_encoded_dev.shape)

(1101, 49)
(1101,)


In [12]:
print(x_test_data_padded.shape)
print(categorical_labels_test.shape)

(2210, 49)
(2210, 5)


In [13]:
from convnets_utils import *

# CNN with random word embeddings

In [14]:
model_1 = get_cnn_rand(200, len(word_index)+1, max_input_lenght, 5)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
history = model_1.fit(x=x_train_data_padded, y=categorical_labels_train, batch_size=50, epochs=15)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
8544/8544 [==============================] - 28s 3ms/step - loss: 0.4455 - acc: 0.8028
Epoch 2/15
8544/8544 [==============================] - 24s 3ms/step - loss: 0.4185 - acc: 0.8109
Epoch 3/15
8544/8544 [==============================] - 24s 3ms/step - loss: 0.4163 - acc: 0.8126: 3s - loss
Epoch 4/15
8544/8544 [==============================] - 23s 3ms/step - loss: 0.4134 - acc: 0.8143
Epoch 5/15
8544/8544 [==============================] - 24s 3ms/step - loss: 0.4104 - acc: 0.8150
Epoch 6/15
8544/8544 [==============================] - 24s 3ms/step - loss: 0.4054 - acc: 0.8195
Epoch 7/15
8544/8544 [==============================] - 24s 3ms/step - loss: 0.3994 - acc: 0.8236
Epoch 8/15
8544/8544 [==============================] - 23s 3ms/step - loss: 0.3871 - acc: 0.8349
Epoch 9/15
8544/8544 [==============================] - 24s 3ms/step - loss: 0.3746 - acc: 0.8427
Epoch 10/15
8544/8544 [==============================] - 26

In [16]:
loss, accuracy = model_1.evaluate(x_test_data_padded, categorical_labels_test, verbose=0)
accuracy

0.8002714871820821

In [17]:
raw_predictions = model_1.predict(x_test_data_padded)
class_predictions = [np.argmax(x) for x in raw_predictions]
print(classification_report(y_test_data_categ, le.inverse_transform(class_predictions)))

               precision    recall  f1-score   support

     negative       0.19      0.19      0.19       405
      neutral       0.52      0.67      0.59      1155
     positive       0.16      0.12      0.14       424
very positive       0.00      0.00      0.00       112
very_negative       0.00      0.00      0.00       114

     accuracy                           0.41      2210
    macro avg       0.18      0.20      0.18      2210
 weighted avg       0.34      0.41      0.37      2210



C:\Users\Xenon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# CNN with pre-trained static word embeddings

In [18]:
embeddings_index = load_fasttext_embeddings()
embeddings_matrix = create_embeddings_matrix(embeddings_index, word_index, 100)
embedding_layer_static = get_embeddings_layer(embeddings_matrix, 'embedding_layer_static', max_input_lenght, trainable=False)
model_2 = get_cnn_pre_trained_embeddings(embedding_layer_static, max_input_lenght, 5)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/dsbatista/resources/glove.6B\\glove.6B.100d.txt'

In [ ]:
history = model_2.fit(x=x_train_data_padded, y=categorical_labels_train, batch_size=50, epochs=15)

In [ ]:
loss, accuracy = model_2.evaluate(x_test_data_padded, categorical_labels_test, verbose=0)
accuracy

In [ ]:
raw_predictions = model_2.predict(x_test_data_padded)
class_predictions = [np.argmax(x) for x in raw_predictions]
print(classification_report(y_test_data_categ, le.inverse_transform(class_predictions)))

# CNN with pre-trained dynamic word embeddings

In [ ]:
embedding_layer_dynamic = get_embeddings_layer(embeddings_matrix, 'embedding_layer_dynamic', 
                                               max_input_lenght, trainable=True)

model_3 = get_cnn_pre_trained_embeddings(embedding_layer_dynamic, max_input_lenght, 5)

In [ ]:
history = model_3.fit(x=x_train_data_padded, y=categorical_labels_train, 
                      batch_size=50, 
                      epochs=5)

In [ ]:
loss, accuracy = model_3.evaluate(x_test_data_padded, categorical_labels_test, verbose=0)
accuracy

In [ ]:
raw_predictions = model_3.predict(x_test_data_padded)
class_predictions = [np.argmax(x) for x in raw_predictions]
print(classification_report(y_test_data_categ, le.inverse_transform(class_predictions)))

# CNN multichanell with pre-trained dynamic and static word embeddings

In [ ]:
model_4 = get_cnn_multichannel(embedding_layer_static, embedding_layer_dynamic, max_input_lenght, 5)

In [ ]:
history = model_4.fit(x=[x_train_data_padded,x_train_data_padded], y=categorical_labels_train, 
                      batch_size=50, 
                      epochs=5,
                      validation_split=0.33)

In [ ]:
loss, accuracy = model_4.evaluate(x=[x_test_data_padded,x_test_data_padded], y=categorical_labels_test, verbose=0)
accuracy

In [ ]:
raw_predictions = model_4.predict(x=[x_test_data_padded,x_test_data_padded])
class_predictions = [np.argmax(x) for x in raw_predictions]
print(classification_report(y_test_data_categ, le.inverse_transform(class_predictions)))